In [1]:
import wget
import os.path
import json


import numpy as np
from numpy.linalg import det, inv
from numpy import array
import random

import ase.db
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
from anneling_super_cell import build_annealing

In [3]:
DATA_PATH = "database"
database_name1 = "c2db-20211702.db"
database_name2 = "c2db-20211702.db"

In [4]:
def get_data_as_pd(path,options,props):
    db = ase.db.connect(path)
    rows = db.select(options)
    data = [ [ x.get(p) for p in props ] for x in rows]
    raw_df = pd.DataFrame( data, columns = props);
    return raw_df 

In [5]:
path = DATA_PATH+database_name1

options = 'is_magnetic=True, thermodynamic_stability_level=3'
props = ["formula","spgnum", "spacegroup","uid","asr_id"] 

raw_df_1 = get_data_as_pd(path,options,props)
raw_df_1 = raw_df_1[ (raw_df_1["spacegroup"]!='P1') & (raw_df_1["spacegroup"]!='Pc')& (raw_df_1["spacegroup"]!='P-1') ];


#Print
print("The number of elements is:",len(list(raw_df_1)) )
print( raw_df_1[["formula","uid"]] )

The number of elements is: 5
Empty DataFrame
Columns: [formula, uid]
Index: []


In [6]:
path = DATA_PATH+database_name2

options = 'thermodynamic_stability_level=3'
props = ["formula","spgnum", "spacegroup","gap","uid"] 

raw_df_2 = get_data_as_pd(path,options,props)
raw_df_2 = raw_df_2[ (raw_df_2["gap"]<=10e-3)& (raw_df_2["spacegroup"]!='P1') & (raw_df_2["spacegroup"]!='Pc')& (raw_df_2["spacegroup"]!='P-1') ];


#Print
print("The number of elements is:",len(list(raw_df_2)) )
print( raw_df_2[["formula","uid","gap"]] )

The number of elements is: 5
Empty DataFrame
Columns: [formula, uid, gap]
Index: []


In [7]:
# get jsons 
def extract_json(uid_list,save_path ="JSONcolection"):
    for uid in uid_list:
        dataurl = 'https://cmrdb.fysik.dtu.dk/c2db/row/'+uid+'/all_data';
        file = save_path+"/"+uid+"_data.json"
        if os.path.isfile(file):
            print("file: ",file, "found")
        else:
            print(wget.download(dataurl, out="./"+save_path))

def extract_structure(uid_list ,save_path="XYZcolection"):
    for uid in uid_list:
        
        #https://cmrdb.fysik.dtu.dk/c2db/row/N2O2V3-358facb64a22/data/structure.json
        dataurl = 'https://cmrdb.fysik.dtu.dk/c2db/row/'+uid+'/data/structure.json';
        print(dataurl)
        file = save_path+"/"+uid+".json"
        dw_path = save_path+"/"+"structure.json"
        if os.path.isfile(file):
            print("file: ",file, "found")
        else:
            print(wget.download(dataurl, out="./"+save_path))
            os.rename(dw_path, file)

In [8]:
print("extract_data1:")
print(raw_df_1["uid"][:1])
extract_structure(raw_df_1["uid"][:10])
print("extract_data2:")
extract_structure(raw_df_1["uid"][:10])

extract_data1:
Series([], Name: uid, dtype: object)
extract_data2:


In [9]:
# Example of good practice in data hadeling 
# uid= "C2-a6735a4a3797"
# with open('XYZcolection/structure.json', 'r') as file:
#     json_data= json.load(file)
# mytype= "__ndarray__";
# datastruc=json_data['1']["cell"]["array"];
# if mytype in datastruc:
#     shape= datastruc[mytype][0]
#     dtype= datastruc[mytype][1]
#     lattice = array(datastruc[mytype][2],dtype=dtype).reshape(shape)


In [10]:
#f = "/Users/voicutu/Documents/GitHub/vdw_evolve/sandbox/XYZcolection/c2db-822.xyz"

def get_lattice_from_structure(f):
    with open(f, 'r') as file:
        json_data= json.load(file)
    mytype= "__ndarray__";
    datastruc=json_data['1']["cell"]["array"];
    if mytype in datastruc:
        shape= datastruc[mytype][0]
        dtype= datastruc[mytype][1]
        lattice = array(datastruc[mytype][2],dtype=dtype).reshape(shape)
    return lattice 

In [11]:
#
up = 99999999
bond =20
model_par = {
    
    'initialTemp': 4, 
    'finalTemp': 0.0002,

    'beta': 10,
    'bounds': [[-2*bond,2*bond] for _ in range(4)],

    'nr_neighbours': 1,
    'step_size': 4,
    'gaussian_sd':3,
    
    'known_min': -up,
    "start_point":[1,5,3,2]
}



In [12]:
uids=["C2-a6735a4a3797","WS2-64090c9845f8"]
extract_structure(uids)

https://cmrdb.fysik.dtu.dk/c2db/row/C2-a6735a4a3797/data/structure.json
file:  XYZcolection/C2-a6735a4a3797.json found
https://cmrdb.fysik.dtu.dk/c2db/row/WS2-64090c9845f8/data/structure.json
file:  XYZcolection/WS2-64090c9845f8.json found


In [13]:
# xyz path 

structure_path = "XYZcolection"#/Users/voicutu/Documents/GitHub/vdw_evolve/sandbox/XYZcolection"
uid_1 = "WS2-64090c9845f8"#"VCl2-1ccb0b28392d"
uid_2 = "C2-a6735a4a3797"#"Mn2Se2-a1d595c25fc5"


path1=structure_path+"/"+uid_1+".json"
path2=structure_path+"/"+uid_2+".json"

# lattices
latice1 = get_lattice_from_structure(path1)
latice2 = get_lattice_from_structure(path2)


cel1 = np.array([[latice1[0][0], latice1[1][0]],
        [latice1[0][1], latice1[1][1]]])

cel2 = np.array([[latice2[0][0], latice2[1][0]],
        [latice2[0][1], latice2[1][1]]])

tA, tB, strain = build_annealing(cel1,cel2, 7, model_par)
new = np.dot(tA, cel1)

print("\ncel1",cel1)
print("cel2",cel2)
print("\n _____\n tA:\n",tA)
print("det:",det(tA))
print("\n _____\n tB:\n",tB)
print("det:",det(tB))
super_cell= np.dot(tA,cel1)
print("\n _____\n Super cell:\n",super_cell)
print("det:",det(super_cell))
print("\n _____\n Strain:\n",strain)
suuper_cell= np.dot(tA,cel1)



# print("\n zero_mat:\n",zero_mat)

temp:4|epoch:0|change:18257373576.858765|value:222647965368.2426
temp:4|epoch:5|change:11367612753445.176|value:36362916568.13632
temp:0.0975609756097561|epoch:0|change:88447123981.24788|value:36362916568.13632
temp:0.0975609756097561|epoch:5|change:47428630004.265945|value:36362916568.13632
temp:0.04938271604938272|epoch:0|change:107936120173.50998|value:36362916568.13632
temp:0.04938271604938272|epoch:5|change:222699464608.32516|value:36362916568.13632
temp:0.03305785123966942|epoch:0|change:71046061377.51659|value:8619380746.234423
temp:0.03305785123966942|epoch:5|change:6047940664.313038|value:8619380746.234423
temp:0.02484472049689441|epoch:0|change:457660806683.80096|value:8619380746.234423
temp:0.02484472049689441|epoch:5|change:145834937876.30585|value:8619380746.234423
temp:0.01990049751243781|epoch:0|change:155393510626.91614|value:8619380746.234423
temp:0.01990049751243781|epoch:5|change:521876695059.4845|value:8619380746.234423
temp:0.016597510373443983|epoch:0|change:11385

temp:0.0019598236158745717|epoch:0|change:165316542268.86685|value:2827717381.6066155
temp:0.0019598236158745717|epoch:5|change:100047410554.4276|value:2827717381.6066155
temp:0.0019221528111484868|epoch:0|change:519921367540.0289|value:2827717381.6066155
temp:0.0019221528111484868|epoch:5|change:303766063494.1836|value:2827717381.6066155
temp:0.0018859028760018863|epoch:0|change:44340965314.89288|value:2827717381.6066155
temp:0.0018859028760018863|epoch:5|change:318033644283.70496|value:2827717381.6066155
temp:0.0018509949097639986|epoch:0|change:1439836406.3477888|value:2827717381.6066155
temp:0.0018509949097639986|epoch:5|change:29218775391.517284|value:2827717381.6066155
temp:0.0018173557473875517|epoch:0|change:314759597396.85583|value:2827717381.6066155
temp:0.0018173557473875517|epoch:5|change:347426756261.0743|value:2827717381.6066155
temp:0.0017849174475680506|epoch:0|change:155230977421.12244|value:2827717381.6066155
temp:0.0017849174475680506|epoch:5|change:307843744183.7315

temp:0.0010098459984852306|epoch:5|change:112418683155.10666|value:114626142.3461247
temp:0.0009997500624843783|epoch:0|change:1335637151577.3298|value:114626142.3461247
temp:0.0009997500624843783|epoch:5|change:2064919427.6834373|value:114626142.3461247
temp:0.0009898539965355103|epoch:0|change:287914516644.90015|value:114626142.3461247
temp:0.0009898539965355103|epoch:5|change:3736555487772.7354|value:114626142.3461247
temp:0.0009801519235481493|epoch:0|change:120653549716.9935|value:114626142.3461247
temp:0.0009801519235481493|epoch:5|change:39634003892.13791|value:114626142.3461247
temp:0.0009706381946129574|epoch:0|change:164316955993.07565|value:114626142.3461247
temp:0.0009706381946129574|epoch:5|change:37007717981.0533|value:114626142.3461247
temp:0.0009613073780341257|epoch:0|change:41324331266.51717|value:114626142.3461247
temp:0.0009613073780341257|epoch:5|change:7963845671876.93|value:114626142.3461247
temp:0.0009521542489883355|epoch:0|change:780848680338.5537|value:114626

temp:0.000671028350947827|epoch:5|change:134505177449.60167|value:114626142.3461247
temp:0.0006665555740709876|epoch:0|change:2069487079220.6416|value:114626142.3461247
temp:0.0006665555740709876|epoch:5|change:776023789784.441|value:114626142.3461247
temp:0.0006621420294653197|epoch:0|change:718652471431.2363|value:114626142.3461247
temp:0.0006621420294653197|epoch:5|change:136015301199.27258|value:114626142.3461247
temp:0.0006577865482650873|epoch:0|change:141193669680.84497|value:114626142.3461247
temp:0.0006577865482650873|epoch:5|change:188687408699.62256|value:114626142.3461247
temp:0.0006534879921581434|epoch:0|change:159098342113.23868|value:114626142.3461247
temp:0.0006534879921581434|epoch:5|change:15509706790.645212|value:114626142.3461247
temp:0.0006492452523940912|epoch:0|change:172100064949.22995|value:114626142.3461247
temp:0.0006492452523940912|epoch:5|change:136409507553.25987|value:114626142.3461247
temp:0.0006450572488308331|epoch:0|change:1308635743413.2827|value:11

temp:0.0005024494410249969|epoch:5|change:7995099198955.12|value:114626142.3461247
temp:0.0004999375078115236|epoch:0|change:87906149568.13445|value:114626142.3461247
temp:0.0004999375078115236|epoch:5|change:194331144703.9647|value:114626142.3461247
temp:0.0004974505658500187|epoch:0|change:183074707959.20908|value:114626142.3461247
temp:0.0004974505658500187|epoch:5|change:687965280647.0443|value:114626142.3461247
temp:0.0004949882440292043|epoch:0|change:106971973267.47067|value:114626142.3461247
temp:0.0004949882440292043|epoch:5|change:338542827739.44525|value:114626142.3461247
temp:0.0004925501785494397|epoch:0|change:102332886873.46278|value:114626142.3461247
temp:0.0004925501785494397|epoch:5|change:312040604935.284|value:114626142.3461247
temp:0.0004901360127435363|epoch:0|change:247135457023.64352|value:114626142.3461247
temp:0.0004901360127435363|epoch:5|change:2393339774286.9014|value:114626142.3461247
temp:0.0004877453969028167|epoch:0|change:85194950838.27621|value:114626

...

temp:0.0003366719973066246|epoch:5|change:68899115208.25903|value:114626142.3461247
temp:0.00033554232027514525|epoch:0|change:116316506231.76389|value:114626142.3461247
temp:0.00033554232027514525|epoch:5|change:5593233034.46745|value:114626142.3461247
temp:0.0003344201989800189|epoch:0|change:183298648222.94736|value:114626142.3461247
temp:0.0003344201989800189|epoch:5|change:97426954744.0252|value:114626142.3461247
temp:0.000333305557870178|epoch:0|change:160289309746.8674|value:114626142.3461247
temp:0.000333305557870178|epoch:5|change:1378505889899.5896|value:114626142.3461247
temp:0.0003321983223984724|epoch:0|change:880380343096.6542|value:114626142.3461247
temp:0.0003321983223984724|epoch:5|change:858313267403.2766|value:114626142.3461247
temp:0.0003310984190050497|epoch:0|change:225753088048.83112|value:114626142.3461247
temp:0.0003310984190050497|epoch:5|change:119272076345.86723|value:114626142.3461247
temp:0.00033000577510106473|epoch:0|change:160539171656.97855|value:11462

temp:0.0002881636769685183|epoch:0|change:57809600888.01753|value:114626142.3461247
temp:0.0002881636769685183|epoch:5|change:108932187132.01266|value:114626142.3461247
temp:0.0002873356799080528|epoch:0|change:9579899535743.742|value:114626142.3461247
temp:0.0002873356799080528|epoch:5|change:19358644367.334267|value:114626142.3461247
temp:0.000286512427476542|epoch:0|change:419934446347.79346|value:114626142.3461247
temp:0.000286512427476542|epoch:5|change:293699477090.54095|value:114626142.3461247
temp:0.0002856938790086424|epoch:0|change:321937257066.69965|value:114626142.3461247
temp:0.0002856938790086424|epoch:5|change:80308571584.36981|value:114626142.3461247
temp:0.00028487999430240027|epoch:0|change:228860906557.7117|value:114626142.3461247
temp:0.00028487999430240027|epoch:5|change:250653434801.74997|value:114626142.3461247
temp:0.0002840707336126697|epoch:0|change:3347357021373.978|value:114626142.3461247
temp:0.0002840707336126697|epoch:5|change:264241368480.32806|value:114

temp:0.0002531485349028542|epoch:5|change:8011758880799.399|value:114626142.3461247
temp:0.0002525093112808534|epoch:0|change:617703258040.1053|value:114626142.3461247
temp:0.0002525093112808534|epoch:5|change:315247325170.4579|value:114626142.3461247
temp:0.00025187330772621366|epoch:0|change:216943656225.34937|value:114626142.3461247
temp:0.00025187330772621366|epoch:5|change:1065943753529.5923|value:114626142.3461247
temp:0.00025124049996859486|epoch:0|change:6886761128284.957|value:114626142.3461247
temp:0.00025124049996859486|epoch:5|change:608444703199.3104|value:114626142.3461247
temp:0.0002506108639809535|epoch:0|change:353630970824.3715|value:114626142.3461247
temp:0.0002506108639809535|epoch:5|change:209250962148.0312|value:114626142.3461247
temp:0.0002499843759765014|epoch:0|change:32910492946.22675|value:114626142.3461247
temp:0.0002499843759765014|epoch:5|change:20139050078026.324|value:114626142.3461247
temp:0.00024936101240571033|epoch:0|change:389968277110.87695|value:1

temp:0.00022521254433871966|epoch:5|change:1605665361199.6355|value:114626142.3461247
temp:0.00022470647716420425|epoch:0|change:93577307698.79922|value:114626142.3461247
temp:0.00022470647716420425|epoch:5|change:4481590473532.453|value:114626142.3461247
temp:0.00022420267922201672|epoch:0|change:5228034477665.278|value:114626142.3461247
temp:0.00022420267922201672|epoch:5|change:289887919019.85284|value:114626142.3461247
temp:0.0002237011352832616|epoch:0|change:108096991228.6827|value:114626142.3461247
temp:0.0002237011352832616|epoch:5|change:29752924082.805138|value:114626142.3461247
temp:0.0002232018302550081|epoch:0|change:811502766873.8064|value:114626142.3461247
temp:0.0002232018302550081|epoch:5|change:128581329723.11829|value:114626142.3461247
temp:0.00022270474917877623|epoch:0|change:658412339912.4111|value:114626142.3461247
temp:0.00022270474917877623|epoch:5|change:267872879441.45474|value:114626142.3461247
temp:0.0002222098772290428|epoch:0|change:739098088855.1521|valu

temp:0.00020282947112215392|epoch:5|change:668340742479.01|value:114626142.3461247
temp:0.00020241890592581334|epoch:0|change:101833287136.04292|value:114626142.3461247
temp:0.00020241890592581334|epoch:5|change:4468175269939.523|value:114626142.3461247
temp:0.00020200999949497489|epoch:0|change:2082512844909.7954|value:114626142.3461247
temp:0.00020200999949497489|epoch:5|change:35020583225.60713|value:114626142.3461247
temp:0.00020160274179728833|epoch:0|change:80344120079.19872|value:114626142.3461247
temp:0.00020160274179728833|epoch:5|change:547048162077.1401|value:114626142.3461247
temp:0.0002011971228811427|epoch:0|change:1950969111537.6575|value:114626142.3461247
temp:0.0002011971228811427|epoch:5|change:134933272879.84775|value:114626142.3461247
temp:0.0002007931328748556|epoch:0|change:194656115491.6402|value:114626142.3461247
temp:0.0002007931328748556|epoch:5|change:279356701916.54266|value:114626142.3461247
temp:0.00020039076198587235|epoch:0|change:45073269722.8006|value:

In [14]:
solutions = {"uid_1":[], "uid_2":[],
             "mat_1":[], "mat_2":[],
             "tA":[],    "tB":[],
             "new":[],   "strainB":[],
             "new_uid":[],       }

In [15]:
structure_path = "/Users/voicutu/Documents/GitHub/vdw_evolve/sandbox/XYZcolection"
saving_spot = 5
to_be_generated = 10
while to_be_generated!=0:
    
    data1= raw_df_1.iloc[random.randinit(0,len(raw_df_1["uid"]))
    data2= raw_df_1.iloc[random.randinit(0,len(raw_df_2["uid"]))
    uid_1 = data1["uid"]
    uid_2 = data2["uid"]
    
    new_uid =uid_1+"_"+uid_2
    
    if new_uid not in solutions["new_uid"]:
        to_be_generated = to_be_generated-1
        
        solutions["new_uid"].append(new_uid)
        solutions["uid_1"].append(uid_1)
        solutions["uid_2"].append(uid_2)
        
        # calculate the super cel 
        
        # xyz path 
        path1=structure_path+"/"+uid_1+".json"
        path2=structure_path+"/"+uid_2+".json"
        
        # lattices
        latice1 = get_lattice_from_structure(path1)
        latice2 = get_lattice_from_structure(path2)


        cel1 = [[latice1[0][0], latice1[1][0]],
                [latice1[0][1], latice1[1][1]]]

        cel2 = [[latice2[0][0], latice2[1][0]],
                [latice2[0][1], latice2[1][1]]]
        
        tA, tB, strain = build_annealing(cel1,cel2, 5, model_par)
        new = np.dot(tA, cel1)
        
        solutions["tA"].append(tA)
        solutions["tB"].append(tB)
        solutions["new"].append(new)
        solutions["strainB"].append(strain)
    
    # save
    if k%saving_spot==0:
        data = pd.DataFrame.from_dict(solutions)
        data.to_csv("SuperCell_"+str(len(solutions["new"])))

print("Done")

SyntaxError: invalid syntax (1642023313.py, line 7)